In [5]:
# Imports
import numpy as np
import pandas as pd

from edinburgh_challenge.constants import police_stations
from edinburgh_challenge.utility import generate_early_shift_distributions
from edinburgh_challenge.models import NaiveModel
from edinburgh_challenge.simulation import *
from edinburgh_challenge.processing import calculate_metric

In [6]:
source = "./data.xlsx"
data = pd.read_excel(source)
data["Time"] = (data["Day"]-1)*24 + data["Hour"]
data.columns = [x.lower() for x in data.columns]

## Formulating the metric

{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 100.0},
 'Mean Response Times': {'Immediate': 88.33043348830911,
  'Prompt': 84.65961030075212,
  'Standard': 88.37365649837112},
 'Mean Deployment Times': {'Immediate': 1.5403225806451613,
  'Prompt': 1.5061196105702366,
  'Standard': 1.4796755725190842},
 'Threshold Compliance': {'Immediate': 100.0,
  'Prompt': 98.95688456189151,
  'Standard': 98.09160305343512},
 'Total Officer Hours': 4422.399999003844,
 'Unresolved Incident Percentage': 0.0
 }

In [81]:
def calculate_simulation_performance(results_dict):
    # Information from the results analysis
    immediate_completion_pct = results_dict["Completion Percentages"]["Immediate"]
    immediate_threshold_pct = results_dict["Threshold Compliance"]["Immediate"]

    prompt_completion_pct = results_dict["Completion Percentages"]["Prompt"]
    prompt_threshold_pct = results_dict["Threshold Compliance"]["Prompt"]

    standard_completion_pct = results_dict["Completion Percentages"]["Standard"]
    standard_threshold_pct = results_dict["Threshold Compliance"]["Standard"]
    
    mean_officer_hours  = results_dict["Mean Officer Hours"]

    # Rescaling these values
    immediate_completion_pct /= 100
    prompt_completion_pct /= 100
    standard_completion_pct /= 100

    immediate_threshold_pct /= 100
    prompt_threshold_pct /= 100
    standard_threshold_pct /= 100

    immediate_incompletion_pct = 1 - immediate_completion_pct
    prompt_incompletion_pct = 1- prompt_completion_pct
    standard_incompletion_pct = 1 - standard_completion_pct
    
    # Calculating the score

    # First factor - Incident resolved within threshold (Scale - 0 to 1)
    incident_within_threshold = (2*immediate_threshold_pct + 1.5*prompt_threshold_pct + 1*standard_threshold_pct)/(4.5)

    # Second factor - Officer utilisation
    # 8 hours per shift, 7 days in the simulation (Scale - 0 to 1)
    officer_utilisation = (mean_officer_hours)/(8*7)

    # Third factor - Unresolved Incidents (Scale - 0 to 1)
    unresolved_incidents = ((6*immediate_incompletion_pct)+ 2*(prompt_incompletion_pct) + 1*(standard_incompletion_pct))/9
    
    # Total scale, (0 to 1)
    performance_metric = 0.8*incident_within_threshold + 0.2*officer_utilisation - unresolved_incidents*0.3
    return performance_metric

In [90]:
shift_distribution = {'Early': {'Station_1': 7, 'Station_2': 0, 'Station_3': 8},
  'Day': {'Station_1': 0, 'Station_2': 0, 'Station_3': 25},
  'Night': {'Station_1': 0, 'Station_2': 11, 'Station_3': 29}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]

simulation = SimulationWithMaxUtilisation(data, ps_coords, shift_distribution, 
                        verbose=0)

In [91]:
naive_model = NaiveModel()

In [92]:
simulation.run(naive_model)

In [93]:
calculate_simulation_performance(simulation.analyze_simulation_results())

0.9943266723146396